In [ ]:

def main():
    svs_file = "data/wsi/gut/522934.svs"
    output_dir = "data/segments"
    level = 1

    # if os.path.exists(output_dir):
    #     shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)
    slide = openslide.OpenSlide(svs_file)
    ds_image = np.array(slide.read_region((0, 0), level, slide.level_dimensions[level]))

    gray_image = cv2.cvtColor(ds_image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY_INV)
    labels = measure.label(binary_image, connectivity=2)
    properties = measure.regionprops(labels)
    del ds_image
    del gray_image
    del binary_image

    W, H = slide.level_dimensions[0]
    w, h = slide.level_dimensions[level]
    scale_factor = W / w
    min_blob_area = 50_000
    large_blobs = [prop for prop in properties if prop.area >= min_blob_area]
    # annotated_image = np.copy(ds_image)
    for i, blob_props in enumerate(large_blobs):
        minr, minc, maxr, maxc = blob_props.bbox
        minr = int(minr * scale_factor)
        maxr = int(maxr * scale_factor)
        minc = int(minc * scale_factor)
        maxc = int(maxc * scale_factor)

        blob_image = np.array(
            slide.read_region((minc, minr), 0, (maxc - minc, maxr - minr)))  # ds_image[minr:maxr, minc:maxc]
        annotated_blob_image = (
            split_image_into_grid(blob_image, 256, 256, os.path.join(output_dir, f"blob_{i:03d}"))
        )

        # cv2.rectangle(annotated_image, (minc, minr), (maxc, maxr), (255, 0, 0), 2)

        display_image(annotated_blob_image, title=f"{annotated_blob_image.shape}")
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
    # display_image(f'Annotated image', annotated_image)


if __name__ == '__main__':
    main()
